# Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, learning_curve,\
train_test_split, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, classification_report, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector

## pipeline stuff

from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn import set_config; set_config(display='diagram')



import warnings 
warnings.filterwarnings('ignore')

In [5]:
from tqdm.auto import tqdm
import itertools

In [6]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

# Preprocessing

In [7]:
train_labels = pd.read_csv("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/train_labels.csv")
df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/train_data.parquet")




In [8]:
df.shape

(5531451, 190)

In [9]:
cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']




# df_def = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_6k_ids_compress.parquet")
# # def_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_20k_ids_compress.parquet")
# df_pay = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/payer_data_20k_ids.parquet")

# target_pay = df_pay.groupby('customer_ID').mean()
# target_def = df_def.groupby('customer_ID').mean()

# target_pay['target'] = int(0)
# target_def['target'] = int(1) 

# target = pd.concat([target_pay, target_def])

# target = target.loc[:,'target']

# target = target.reset_index()

# df = pd.concat([df_pay, df_def])



In [ ]:
### Fancy features

df["c_PD_239"]=df["D_39"]/(df["P_2"]*(-1)+0.0001)
df["c_PB_29"]=df["P_2"]*(-1)/(df["B_9"]*(1)+0.0001)
df["c_PR_21"]=df["P_2"]*(-1)/(df["R_1"]+0.0001)

df["c_BBBB"]=(df["B_9"]+0.001)/(df["B_23"]+df["B_3"]+0.0001)
df["c_BBBB1"]=(df["B_33"]*(-1))+(df["B_18"]*(-1)+df["S_25"]*(1)+0.0001)
df["c_BBBB2"]=(df["B_19"]+df["B_20"]+df["B_4"]+0.0001)

df["c_RRR0"]=(df["R_3"]+0.001)/(df["R_2"]+df["R_4"]+0.0001)
df["c_RRR1"]=(df["D_62"]+0.001)/(df["D_112"]+df["R_27"]+0.0001)

df["c_PD_348"]=df["D_48"]/(df["P_3"]+0.0001)
df["c_PD_355"]=df["D_55"]/(df["P_3"]+0.0001)

df["c_PD_439"]=df["D_39"]/(df["P_4"]+0.0001)
df["c_PB_49"]=df["B_9"]/(df["P_4"]+0.0001)
df["c_PR_41"]=df["R_1"]/(df["P_4"]+0.0001)




In [18]:
## get all feature names, except customer_ID and dates
features = df.drop(columns=[
    'customer_ID', 
    'S_2'
], axis = 1).columns.to_list() 

num_features = [feature for feature in features if feature not in cat_vars]


In [19]:

train_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last']) # give summary statistics for each numerical feature
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns] # join the column name tuples to a single name
train_num_agg.reset_index(inplace = True) # get the customer_ID in as a column again and reset index


In [20]:
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

In [21]:
num_features

['P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_42',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'B_6',
 'B_7',
 'B_8',
 'D_50',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'D_53',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'D_56',
 'B_13',
 'R_5',
 'D_58',
 'S_9',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_65',
 'B_16',
 'B_17',
 'B_18',
 'B_19',
 'B_20',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'D_73',
 'P_4',
 'D_74',
 'D_75',
 'D_76',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'R_9',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'D_82',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_29',
 'S_18',
 'D_86',
 'D_87',
 'R_17',
 'R_18',
 'D_88',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 

In [22]:
train_diff = get_difference(df, num_features)



  0%|          | 0/458913 [00:00<?, ?it/s]

In [27]:
## categorical feature aggregation
train_cat_agg = df.groupby("customer_ID")[cat_vars].agg(['count', 'last', 'nunique']) # give summary statistics for each categrocial feature


In [28]:
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns] # join the column name tuples to a single name


In [29]:
train_cat_agg.reset_index(inplace = True) # get the customer_ID in as a column again and reset index




In [30]:
df_agg = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')

# df_agg = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(target, how = 'inner', on = 'customer_ID')



In [31]:
df_agg

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_137_diff1,D_138_diff1,D_139_diff1,D_140_diff1,D_141_diff1,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.010704,0.024444,0.001082,0.091505,...,NaN,NaN,0.005623,0.002258,0.002516,NaN,0.004712,-0.003376,0.004328,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,0.215205,0.199150,0.002224,0.567403,...,NaN,NaN,-0.003678,0.002852,0.003050,NaN,-0.004988,0.000641,0.004930,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.004181,0.002758,0.000802,0.009704,...,NaN,NaN,-0.000729,-0.002921,-0.002363,NaN,-0.002579,-0.006491,-0.002161,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,0.048862,0.088466,0.000660,0.268476,...,NaN,NaN,-0.006608,-0.000744,0.000454,NaN,-0.001686,0.000741,-0.003864,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891679,0.042325,0.805045,0.940382,0.871900,0.004644,0.002882,0.000030,0.008680,...,NaN,NaN,0.001035,0.001209,-0.001516,NaN,0.003177,0.000618,-0.001154,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.848750,0.041969,0.730505,0.895553,0.844229,0.119236,0.195278,0.002237,0.589866,...,NaN,NaN,-0.001804,-0.008424,0.003578,NaN,-0.006348,-0.002011,-0.000751,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.859327,0.012425,0.831279,0.868121,0.831279,0.066421,0.123937,0.007327,0.478106,...,NaN,NaN,-0.003491,0.000933,-0.006737,NaN,0.003188,-0.000736,0.000247,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.786838,0.014312,0.756983,0.802953,0.800522,0.221548,0.182743,0.000223,0.536278,...,NaN,NaN,-0.000125,-0.006361,-0.006879,NaN,-0.001689,-0.007454,-0.004471,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.804454,0.037442,0.754129,0.856981,0.754129,0.030878,0.081779,0.001332,0.300991,...,NaN,NaN,0.005500,-0.000682,0.008609,0.011276,0.008579,-0.001823,-0.004206,1


In [38]:
df_agg.to_parquet('/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/df_train_agg.parquet')


In [41]:
df_agg.dtypes

customer_ID              object
P_2_mean                float32
P_2_std                 float64
P_2_min                 float32
P_2_max                 float32
                         ...   
D_141_last_mean_diff    float32
D_142_last_mean_diff    float32
D_143_last_mean_diff    float32
D_144_last_mean_diff    float32
D_145_last_mean_diff    float32
Length: 1274, dtype: object

In [42]:
del train_num_agg, train_cat_agg, train_diff
import gc
gc.collect()

NameError: name 'train_num_agg' is not defined

In [33]:
cat_features = [f"{cf}_last" for cf in cat_vars]
encoder = OrdinalEncoder()
df_agg[cat_features] = encoder.fit_transform(df_agg[cat_features])
num_cols_mean = [col for col in df_agg.columns if 'mean' in col]
num_cols_last = [col for col in df_agg.columns if 'last' in col and col not in cat_features]


for col in range(len(num_cols_last)):
    try:
        df_agg[f'{num_cols_last[col]}_mean_diff'] = df_agg[num_cols_last[col]] - df_agg[num_cols_mean[col]]
    except:
        pass
    
    

    

# Amex metric

In [34]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    ## TWEAK
    y_pred = pd.Series(y_pred, index=y_true.index)

    y_true = pd.DataFrame(y_true)
    y_pred = pd.DataFrame(y_pred)

    y_true = y_true.rename(columns={y_true.columns[0]:'target'})
    y_pred = y_pred.rename(columns={y_pred.columns[0]:'prediction'})
    ##

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)


In [45]:
amex_metric_scorer = make_scorer(amex_metric)

## dict of scoring metrics one might want to pass into cross validation
scorings = {'recall':'recall',
            'f1':'f1',
           'amex': amex_metric_scorer}

print('done ✅')

done ✅


# Modelling cat boost reloadeddddd

In [49]:
import optuna
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier, XGBRegressor
import catboost as ctb
from sklearn.model_selection import GridSearchCV, KFold 
import xgboost as xgb
print('XGB Version',xgb.__version__)

print('done ✅')



XGB Version 2.0.0-dev
done ✅


In [50]:
X = df_agg.drop(columns=['customer_ID', 'target'])
y = df_agg['target']

X_train, X_val, y_train, y_val = train_test_split(X, y)


In [56]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X,y, test_size=0.3)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "7gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    gbm = ctb.CatBoostClassifier(**param)
    gbm = ctb.CatBoostClassifier(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    amex = amex_metric(valid_y, pred_labels)
    return amex


In [57]:
%%time
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5, timeout=600)

[I 2022-09-06 16:56:42,270] A new study created in memory with name: no-name-47e3b0a0-292f-468e-bddc-a410e7dd9ada
[I 2022-09-06 16:58:00,488] Trial 0 finished with value: 0.5766759196256442 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.024032045791366613, 'depth': 3, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.713658882016812}. Best is trial 0 with value: 0.5766759196256442.
[I 2022-09-06 17:00:42,709] Trial 1 finished with value: 0.5653895375030693 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.07375244859538782, 'depth': 4, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.544341383589298}. Best is trial 0 with value: 0.5766759196256442.
[I 2022-09-06 17:02:45,054] Trial 2 finished with value: 0.5887793428088335 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.0320351569164368, 'depth': 5, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is tri

CPU times: user 1h 5min 51s, sys: 6min 6s, total: 1h 11min 58s
Wall time: 12min 42s


In [58]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 5
Best trial:
  Value: 0.5935053025899325
  Params: 
    objective: CrossEntropy
    colsample_bylevel: 0.098180934090189
    depth: 8
    boosting_type: Ordered
    bootstrap_type: MVS


In [59]:
param_dist = { }
for key, value in trial.params.items():
    param_dist[key]=value
param_dist


{'objective': 'CrossEntropy',
 'colsample_bylevel': 0.098180934090189,
 'depth': 8,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'MVS'}

In [61]:
gbm = ctb.CatBoostClassifier(**param_dist,iterations=5000)

gbm.fit(X_train, y_train,
    # evaluate loss at each iteration
    eval_set=[(X_train,y_train), (X_val, y_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=100
)


0:	learn: 0.6522757	test: 0.6522757	test1: 0.6523289	best: 0.6523289 (0)	total: 397ms	remaining: 33m 3s
1:	learn: 0.6172510	test: 0.6172510	test1: 0.6173740	best: 0.6173740 (1)	total: 763ms	remaining: 31m 45s
2:	learn: 0.5867449	test: 0.5867449	test1: 0.5869454	best: 0.5869454 (2)	total: 1.11s	remaining: 30m 50s
3:	learn: 0.5578420	test: 0.5578420	test1: 0.5581443	best: 0.5581443 (3)	total: 1.47s	remaining: 30m 39s
4:	learn: 0.5311208	test: 0.5311208	test1: 0.5315047	best: 0.5315047 (4)	total: 1.91s	remaining: 31m 53s
5:	learn: 0.5090991	test: 0.5090991	test1: 0.5095627	best: 0.5095627 (5)	total: 2.29s	remaining: 31m 44s
6:	learn: 0.4876354	test: 0.4876354	test1: 0.4881910	best: 0.4881910 (6)	total: 2.63s	remaining: 31m 13s
7:	learn: 0.4692803	test: 0.4692803	test1: 0.4699245	best: 0.4699245 (7)	total: 2.97s	remaining: 30m 54s
8:	learn: 0.4512288	test: 0.4512288	test1: 0.4519592	best: 0.4519592 (8)	total: 3.31s	remaining: 30m 33s
9:	learn: 0.4356687	test: 0.4356687	test1: 0.4364309	bes

77:	learn: 0.2397771	test: 0.2397771	test1: 0.2426428	best: 0.2426428 (77)	total: 27.8s	remaining: 29m 14s
78:	learn: 0.2394235	test: 0.2394235	test1: 0.2423024	best: 0.2423024 (78)	total: 28.2s	remaining: 29m 15s
79:	learn: 0.2391112	test: 0.2391112	test1: 0.2420087	best: 0.2420087 (79)	total: 28.5s	remaining: 29m 13s
80:	learn: 0.2387703	test: 0.2387703	test1: 0.2416792	best: 0.2416792 (80)	total: 28.9s	remaining: 29m 13s
81:	learn: 0.2384383	test: 0.2384383	test1: 0.2413695	best: 0.2413695 (81)	total: 29.3s	remaining: 29m 14s
82:	learn: 0.2380755	test: 0.2380755	test1: 0.2410277	best: 0.2410277 (82)	total: 29.6s	remaining: 29m 14s
83:	learn: 0.2378052	test: 0.2378052	test1: 0.2407703	best: 0.2407703 (83)	total: 30s	remaining: 29m 14s
84:	learn: 0.2375343	test: 0.2375343	test1: 0.2405318	best: 0.2405318 (84)	total: 30.4s	remaining: 29m 16s
85:	learn: 0.2372149	test: 0.2372149	test1: 0.2402186	best: 0.2402186 (85)	total: 30.7s	remaining: 29m 16s
86:	learn: 0.2369613	test: 0.2369613	te

154:	learn: 0.2265852	test: 0.2265852	test1: 0.2307820	best: 0.2307820 (154)	total: 55.4s	remaining: 28m 52s
155:	learn: 0.2265117	test: 0.2265117	test1: 0.2307211	best: 0.2307211 (155)	total: 55.8s	remaining: 28m 52s
156:	learn: 0.2264287	test: 0.2264287	test1: 0.2306585	best: 0.2306585 (156)	total: 56.1s	remaining: 28m 51s
157:	learn: 0.2263557	test: 0.2263557	test1: 0.2305930	best: 0.2305930 (157)	total: 56.5s	remaining: 28m 51s
158:	learn: 0.2262895	test: 0.2262895	test1: 0.2305431	best: 0.2305431 (158)	total: 56.8s	remaining: 28m 50s
159:	learn: 0.2262001	test: 0.2262001	test1: 0.2304612	best: 0.2304612 (159)	total: 57.2s	remaining: 28m 49s
160:	learn: 0.2261232	test: 0.2261232	test1: 0.2303938	best: 0.2303938 (160)	total: 57.5s	remaining: 28m 49s
161:	learn: 0.2260318	test: 0.2260318	test1: 0.2303177	best: 0.2303177 (161)	total: 57.9s	remaining: 28m 49s
162:	learn: 0.2259227	test: 0.2259227	test1: 0.2302301	best: 0.2302301 (162)	total: 58.2s	remaining: 28m 48s
163:	learn: 0.22584

230:	learn: 0.2217964	test: 0.2217964	test1: 0.2272113	best: 0.2272113 (230)	total: 1m 24s	remaining: 29m 4s
231:	learn: 0.2217540	test: 0.2217540	test1: 0.2271824	best: 0.2271824 (231)	total: 1m 24s	remaining: 29m 4s
232:	learn: 0.2217215	test: 0.2217215	test1: 0.2271545	best: 0.2271545 (232)	total: 1m 25s	remaining: 29m 3s
233:	learn: 0.2216692	test: 0.2216692	test1: 0.2271242	best: 0.2271242 (233)	total: 1m 25s	remaining: 29m 3s
234:	learn: 0.2216196	test: 0.2216196	test1: 0.2270843	best: 0.2270843 (234)	total: 1m 25s	remaining: 29m 2s
235:	learn: 0.2215637	test: 0.2215637	test1: 0.2270535	best: 0.2270535 (235)	total: 1m 26s	remaining: 29m 2s
236:	learn: 0.2215123	test: 0.2215123	test1: 0.2270194	best: 0.2270194 (236)	total: 1m 26s	remaining: 29m 2s
237:	learn: 0.2214677	test: 0.2214677	test1: 0.2269894	best: 0.2269894 (237)	total: 1m 27s	remaining: 29m 1s
238:	learn: 0.2214330	test: 0.2214330	test1: 0.2269671	best: 0.2269671 (238)	total: 1m 27s	remaining: 29m 1s
239:	learn: 0.22136

305:	learn: 0.2186908	test: 0.2186908	test1: 0.2251849	best: 0.2251849 (305)	total: 1m 51s	remaining: 28m 36s
306:	learn: 0.2186584	test: 0.2186584	test1: 0.2251701	best: 0.2251701 (306)	total: 1m 52s	remaining: 28m 36s
307:	learn: 0.2186223	test: 0.2186223	test1: 0.2251457	best: 0.2251457 (307)	total: 1m 52s	remaining: 28m 36s
308:	learn: 0.2185928	test: 0.2185928	test1: 0.2251317	best: 0.2251317 (308)	total: 1m 53s	remaining: 28m 36s
309:	learn: 0.2185605	test: 0.2185605	test1: 0.2251134	best: 0.2251134 (309)	total: 1m 53s	remaining: 28m 35s
310:	learn: 0.2185219	test: 0.2185219	test1: 0.2250987	best: 0.2250987 (310)	total: 1m 53s	remaining: 28m 35s
311:	learn: 0.2184828	test: 0.2184828	test1: 0.2250753	best: 0.2250753 (311)	total: 1m 54s	remaining: 28m 35s
312:	learn: 0.2184386	test: 0.2184386	test1: 0.2250512	best: 0.2250512 (312)	total: 1m 54s	remaining: 28m 35s
313:	learn: 0.2184039	test: 0.2184039	test1: 0.2250252	best: 0.2250252 (313)	total: 1m 54s	remaining: 28m 35s
314:	learn

...

2894:	learn: 0.1694467	test: 0.1694467	test1: 0.2177388	best: 0.2177374 (2892)	total: 16m 11s	remaining: 11m 46s
2895:	learn: 0.1694389	test: 0.1694389	test1: 0.2177380	best: 0.2177374 (2892)	total: 16m 11s	remaining: 11m 46s
2896:	learn: 0.1694210	test: 0.1694210	test1: 0.2177388	best: 0.2177374 (2892)	total: 16m 12s	remaining: 11m 45s
2897:	learn: 0.1694036	test: 0.1694036	test1: 0.2177386	best: 0.2177374 (2892)	total: 16m 12s	remaining: 11m 45s
2898:	learn: 0.1693889	test: 0.1693889	test1: 0.2177415	best: 0.2177374 (2892)	total: 16m 12s	remaining: 11m 45s
2899:	learn: 0.1693772	test: 0.1693772	test1: 0.2177414	best: 0.2177374 (2892)	total: 16m 13s	remaining: 11m 44s
2900:	learn: 0.1693687	test: 0.1693687	test1: 0.2177416	best: 0.2177374 (2892)	total: 16m 13s	remaining: 11m 44s
2901:	learn: 0.1693470	test: 0.1693470	test1: 0.2177436	best: 0.2177374 (2892)	total: 16m 13s	remaining: 11m 44s
2902:	learn: 0.1693371	test: 0.1693371	test1: 0.2177425	best: 0.2177374 (2892)	total: 16m 13s	re

2967:	learn: 0.1684822	test: 0.1684822	test1: 0.2177379	best: 0.2177354 (2910)	total: 16m 31s	remaining: 11m 18s
2968:	learn: 0.1684724	test: 0.1684724	test1: 0.2177385	best: 0.2177354 (2910)	total: 16m 31s	remaining: 11m 18s
2969:	learn: 0.1684579	test: 0.1684579	test1: 0.2177337	best: 0.2177337 (2969)	total: 16m 31s	remaining: 11m 18s
2970:	learn: 0.1684446	test: 0.1684446	test1: 0.2177335	best: 0.2177335 (2970)	total: 16m 32s	remaining: 11m 17s
2971:	learn: 0.1684308	test: 0.1684308	test1: 0.2177327	best: 0.2177327 (2971)	total: 16m 32s	remaining: 11m 17s
2972:	learn: 0.1684211	test: 0.1684211	test1: 0.2177323	best: 0.2177323 (2972)	total: 16m 32s	remaining: 11m 17s
2973:	learn: 0.1684045	test: 0.1684045	test1: 0.2177334	best: 0.2177323 (2972)	total: 16m 33s	remaining: 11m 16s
2974:	learn: 0.1683871	test: 0.1683871	test1: 0.2177297	best: 0.2177297 (2974)	total: 16m 33s	remaining: 11m 16s
2975:	learn: 0.1683706	test: 0.1683706	test1: 0.2177294	best: 0.2177294 (2975)	total: 16m 33s	re

3040:	learn: 0.1674997	test: 0.1674997	test1: 0.2177261	best: 0.2177178 (3034)	total: 16m 51s	remaining: 10m 51s
3041:	learn: 0.1674854	test: 0.1674854	test1: 0.2177263	best: 0.2177178 (3034)	total: 16m 51s	remaining: 10m 51s
3042:	learn: 0.1674762	test: 0.1674762	test1: 0.2177247	best: 0.2177178 (3034)	total: 16m 51s	remaining: 10m 50s
3043:	learn: 0.1674676	test: 0.1674676	test1: 0.2177234	best: 0.2177178 (3034)	total: 16m 52s	remaining: 10m 50s
3044:	learn: 0.1674535	test: 0.1674535	test1: 0.2177230	best: 0.2177178 (3034)	total: 16m 52s	remaining: 10m 50s
3045:	learn: 0.1674397	test: 0.1674397	test1: 0.2177212	best: 0.2177178 (3034)	total: 16m 52s	remaining: 10m 49s
3046:	learn: 0.1674253	test: 0.1674253	test1: 0.2177183	best: 0.2177178 (3034)	total: 16m 52s	remaining: 10m 49s
3047:	learn: 0.1674121	test: 0.1674121	test1: 0.2177182	best: 0.2177178 (3034)	total: 16m 53s	remaining: 10m 49s
3048:	learn: 0.1673990	test: 0.1673990	test1: 0.2177153	best: 0.2177153 (3048)	total: 16m 53s	re

3113:	learn: 0.1665198	test: 0.1665198	test1: 0.2176753	best: 0.2176716 (3108)	total: 17m 10s	remaining: 10m 24s
3114:	learn: 0.1664989	test: 0.1664989	test1: 0.2176740	best: 0.2176716 (3108)	total: 17m 11s	remaining: 10m 24s
3115:	learn: 0.1664870	test: 0.1664870	test1: 0.2176751	best: 0.2176716 (3108)	total: 17m 11s	remaining: 10m 23s
3116:	learn: 0.1664708	test: 0.1664708	test1: 0.2176733	best: 0.2176716 (3108)	total: 17m 11s	remaining: 10m 23s
3117:	learn: 0.1664568	test: 0.1664568	test1: 0.2176757	best: 0.2176716 (3108)	total: 17m 12s	remaining: 10m 23s
3118:	learn: 0.1664431	test: 0.1664431	test1: 0.2176795	best: 0.2176716 (3108)	total: 17m 12s	remaining: 10m 22s
3119:	learn: 0.1664361	test: 0.1664361	test1: 0.2176772	best: 0.2176716 (3108)	total: 17m 12s	remaining: 10m 22s
3120:	learn: 0.1664264	test: 0.1664264	test1: 0.2176774	best: 0.2176716 (3108)	total: 17m 12s	remaining: 10m 22s
3121:	learn: 0.1664043	test: 0.1664043	test1: 0.2176768	best: 0.2176716 (3108)	total: 17m 13s	re

3186:	learn: 0.1655837	test: 0.1655837	test1: 0.2176700	best: 0.2176553 (3154)	total: 17m 30s	remaining: 9m 57s
3187:	learn: 0.1655692	test: 0.1655692	test1: 0.2176695	best: 0.2176553 (3154)	total: 17m 30s	remaining: 9m 57s
3188:	learn: 0.1655617	test: 0.1655617	test1: 0.2176663	best: 0.2176553 (3154)	total: 17m 31s	remaining: 9m 57s
3189:	learn: 0.1655499	test: 0.1655499	test1: 0.2176656	best: 0.2176553 (3154)	total: 17m 31s	remaining: 9m 56s
3190:	learn: 0.1655363	test: 0.1655363	test1: 0.2176649	best: 0.2176553 (3154)	total: 17m 31s	remaining: 9m 56s
3191:	learn: 0.1655174	test: 0.1655174	test1: 0.2176685	best: 0.2176553 (3154)	total: 17m 31s	remaining: 9m 56s
3192:	learn: 0.1655006	test: 0.1655006	test1: 0.2176684	best: 0.2176553 (3154)	total: 17m 32s	remaining: 9m 55s
3193:	learn: 0.1654942	test: 0.1654942	test1: 0.2176665	best: 0.2176553 (3154)	total: 17m 32s	remaining: 9m 55s
3194:	learn: 0.1654839	test: 0.1654839	test1: 0.2176661	best: 0.2176553 (3154)	total: 17m 32s	remaining:

In [63]:
y_pred = gbm.predict_proba(X_val)[:,1]


In [64]:
amex_metric(y_val, y_pred)

0.791419807651732

In [65]:
import pickle

In [66]:
pickle.dump(gbm, open('../pickles/cat_boost_reloaded_079', 'wb'))